### 코드 짜보기
> * 풍속센서, 수위센서 모두 사용
> * 모델1,2 둘 다 사용하는 가정하에 만들었습니다
> * 빼기로 한 부분이 있다면 제외하시고 사용하면 될 것 같아요

In [2]:
import joblib

list_windspeed = []
list_wave = []
list_hpa = []

model1 = joblib.load('./model/Random_forest_model_01.pkl') # 모델을 다시 로드(네트워크 + weight)
model2 = joblib.load('./model/XGBoost_minmax_model02.pkl') # 모델을 다시 로드(네트워크 + weight)

scaler1 = joblib.load('./scaler/MinMax_model_01.pkl')
scaler2 = joblib.load('./scaler/MinMax_model_02_storm.pkl')
# scaler2_1 = joblib.load('./scaler/StandardScaler_model_02.pkl')
# scaler2_2 = joblib.load('./scaler/Polynomial_model_02.pkl')

c:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please r

In [3]:
import time
import numpy as np
import paho.mqtt.client as paho
import json
import pandas as pd
from collections import OrderedDict
import joblib

# ip주소 바꾸기
broker="192.168.0.62"
HOST="192.168.0.62"

# 모델1 예측수행 함수
def predictOfModel1(WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA):
    # 센서를 통해 받은 데이터로 예측을 하려면 모델에 맞는 데이터프레임 형태로 만들어 줘야 할 것 같음
    X_data = pd.DataFrame([[WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA]], columns=['WIND_SPEED', 'GUST_WIND', 'SIG_WAVE', 'MAX_WAVE', 'HPA'])
    scaled_data = scaler1.transform(X_data) # MinMax
    pred1 = model1.predict(scaled_data)
    pred1 = pred1[0]
    
    print("predict1 =", pred1)
    return pred1

# 모델2 예측수행 함수
def predictOfModel2(WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA, pred1):
    # 센서를 통해 받은 데이터로 예측을 하려면 모델에 맞는 데이터프레임 형태로 만들어 줘야 할 것 같음
    # X_data = pd.DataFrame([[WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA, pred1]], columns=['WIND_SPEED', 'GUST_WIND', 'SIG_WAVE', 'MAX_WAVE', 'HPA', '특보강도'])
    X_data = pd.DataFrame([[WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA, pred1]], columns=['WIND_SPEED', 'GUST_WIND', 'SIG_WAVE', 'MAX_WAVE', 'HPA', '특보강도'])
    scaled_data = scaler2.transform(X_data)
    # scaled_data['특보강도'] = pred1
    pred2 = model2.predict(scaled_data)
    pred2 = pred2.astype('float64')
    pred2 = pred2[0]
    
    print("predict2 =", pred2)
    return pred2

def makeJSON1(pred1):
    data = OrderedDict()
    data['Predict1'] = pred1
    result = json.dumps(data)
    print("predict message = ", result)
    return result
    
def makeJSON2(pred1, pred2):
    data = OrderedDict()
    data['Predict1'] = pred1
    data['Predict2'] = pred2
    result = json.dumps(data)
    print("predict message = ", result)
    return result

def readJSON(message):
    recvData = str(message.payload.decode("utf-8"))
    print("received message =", recvData)
    jsonData = json.loads(recvData) #json 데이터를 dict형으로 파싱

    return jsonData['Wind'],jsonData['Wave'],jsonData['Hpa']

#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    
    windspeed,wave,hpa = readJSON(message)    #NodeMCU로부터 받은 센서 메시지 파싱 
    # ex) 반복문으로 1분동안 readJson을 통해 얻은 데이터를 리스트로 저장 
    #     -> 풍속데이터 n개가 있는 리스트 하나, 파고데이터 n개가 있는 리스트 하나
    #     -> 풍속: 풍속리스트의 평균
    #        최대순간풍속: 풍속리스트의 최대값
    #        유의파고: 파고리스트의 상위 n/3개 값의 평균값
    #        최대파고: 파고리스트의 최대값

    global list_windspeed, list_wave, list_hpa

    list_windspeed.append(windspeed)
    list_wave.append(wave)
    list_hpa.append(hpa)

    if len(list_windspeed) == 6 : # 60개의 데이터가 쌓이면 실행
        WIND_SPEED = np.mean(list_windspeed)
        GUST_WIND = max(list_windspeed)
        SIG_WAVE = np.mean(sorted(list_wave)[-2:]) # 파고리스트 오름차순 정렬, 상위 20개의 값을 리스트로 반환, 평균계산
        MAX_WAVE = max(list_wave)
        HPA = np.mean(list_hpa)

        # 리스트 초기화
        list_windspeed = []
        list_wave = []
        list_hpa = []

        pred1 = predictOfModel1(WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA) #머신러닝/딥러닝 모델 결과
        if pred1 == 0 : # 모델1의 결과값이 0일때
            preJSON = makeJSON1(pred1) #NodeMCU로 보낼 메시지 만들기 
            client.publish('Model',preJSON) #publish

        elif pred1 != 0 : # 모델1의 결과값이 1,2일때
            pred2 = predictOfModel2(WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA, pred1) # 모델2 예측 수행 0,1 -> ok    2 -> ?
            preJSON = makeJSON2(pred1, pred2) #NodeMCU로 보낼 메시지 만들기  
            client.publish('Model',preJSON) #publish

# 모델1 : WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA -> 특보강도=[0 or 1 or 2]
# 모델2 : WIND_SPEED, GUST_WIND, SIG_WAVE, MAX_WAVE, HPA, 특보강도[0,1,2] -> 남은시간

client = paho.Client()
client.on_message=on_message

print("connecting to broker ",broker)
client.connect(HOST)#connect

print("subscribing ")
client.subscribe("Sensor")#subscribe

try:
    client.loop_forever()
except KeyboardInterrupt:
    print("disconnect.")


connecting to broker  192.168.0.62
subscribing 
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":4,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}


c:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


predict1 = 0.0
predict message =  {"Predict1": 0.0}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":2,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
predict1 = 0.0
predict message =  {"Predict1": 0.0}


c:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


received message = {"Wind":2,"Wave":0.200000003,"Hpa":1038}
received message = {"Wind":3,"Wave":0.200000003,"Hpa":1038}
received message = {"Wind":4,"Wave":0.200000003,"Hpa":1038}
received message = {"Wind":3,"Wave":0.100000001,"Hpa":1038}
received message = {"Wind":4,"Wave":0,"Hpa":1038}
received message = {"Wind":3,"Wave":0,"Hpa":1038}
predict1 = 0.0
predict message =  {"Predict1": 0.0}


...

received message = {"Wind":15,"Wave":0,"Hpa":980}
received message = {"Wind":15,"Wave":7.599999905,"Hpa":980}
received message = {"Wind":9,"Wave":6.599999905,"Hpa":980}
received message = {"Wind":6,"Wave":6.199999809,"Hpa":980}
received message = {"Wind":5,"Wave":6.300000191,"Hpa":980}
received message = {"Wind":11,"Wave":6.400000095,"Hpa":980}
predict1 = 2.0
predict2 = 32.40271759033203
predict message =  {"Predict1": 2.0, "Predict2": 32.40271759033203}


c:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


received message = {"Wind":14,"Wave":6.400000095,"Hpa":980}
received message = {"Wind":14,"Wave":5.400000095,"Hpa":980}
received message = {"Wind":16,"Wave":4.599999905,"Hpa":980}
received message = {"Wind":15,"Wave":3.700000048,"Hpa":980}
received message = {"Wind":16,"Wave":4.199999809,"Hpa":980}
received message = {"Wind":16,"Wave":6.400000095,"Hpa":980}
predict1 = 2.0
predict2 = 22.334611892700195
predict message =  {"Predict1": 2.0, "Predict2": 22.334611892700195}


c:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


received message = {"Wind":14,"Wave":0,"Hpa":980}
received message = {"Wind":10,"Wave":0.100000001,"Hpa":980}
received message = {"Wind":1,"Wave":0,"Hpa":980}
received message = {"Wind":0,"Wave":0,"Hpa":980}
disconnect.
